In [365]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
from scipy import stats
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [366]:
data_path = ("https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv"
            )
crime_df = pd.read_csv(data_path, header=1, index_col=False, skiprows=[0, 1, 2], skipfooter=3)

#Clean up the column names
crime_df.columns=['City','Population', 'Violent crime', 'Murder', 'Rape revised', 'Rape legacy', 'Robbery', 
         'Aggravated assault','Property crime', 'Burglary', 'Larceny', 'Motor vehicle theft', 'Arson']

#Turn the Population strings into integers
crime_df['Population'] = crime_df['Population'].str.replace(',', '')
crime_df['Population'] = crime_df['Population'].astype(int)
#Turn the Robbery strings into integers
crime_df['Robbery'] = crime_df['Robbery'].str.replace(',', '')
crime_df['Robbery'] = crime_df['Robbery'].astype(int)
#Turn the Violent crime strings into integers
crime_df['Violent crime'] = crime_df['Violent crime'].str.replace(',', '')
crime_df['Violent crime'] = crime_df['Violent crime'].astype(int)
#Turn the Aggravated Assault strings into integers
crime_df['Aggravated assault'] = crime_df['Aggravated assault'].str.replace(',', '')
crime_df['Aggravated assault'] = crime_df['Aggravated assault'].astype(int)
#Turn the Property crime strings into integers
crime_df['Property crime'] = crime_df['Property crime'].str.replace(',', '')
crime_df['Property crime'] = crime_df['Property crime'].astype(int)
#Turn the Burglary strings into integers
crime_df['Burglary'] = crime_df['Burglary'].str.replace(',', '')
crime_df['Burglary'] = crime_df['Burglary'].astype(int)
#Turn the Larceny strings into integers
crime_df['Larceny'] = crime_df['Larceny'].str.replace(',', '')
crime_df['Larceny'] = crime_df['Larceny'].astype(int)
#Turn the Motor vehicle theft strings into integers
crime_df['Motor vehicle theft'] = crime_df['Motor vehicle theft'].str.replace(',', '')
crime_df['Motor vehicle theft'] = crime_df['Motor vehicle theft'].astype(int)

#Combine the Rape Scenarios into one category
crime_df['Rape legacy'] = crime_df['Rape legacy'].str.replace(',', '')
crime_df['Rape legacy'] = crime_df['Rape legacy'].astype(int)
crime_df['Rape revised'] = crime_df['Rape revised'].replace([None], 0).astype(int)
crime_df['Rape legacy'] = crime_df['Rape legacy'].replace([None], 0)
crime_df['Rape combined'] = crime_df['Rape revised'] + crime_df['Rape legacy']

crime_df['Arson'] = crime_df['Arson'].replace([None], 0).astype(int)

crime_df.head(500)

/Users/bryancorder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


City  Population  Violent crime  Murder  \
0                 Adams Village        1861              0       0   
1      Addison Town and Village        2577              3       0   
2                 Akron Village        2846              3       0   
3                        Albany       97956            791       8   
4                Albion Village        6388             23       0   
5                Alfred Village        4089              5       0   
6              Allegany Village        1781              3       0   
7                  Amherst Town      118296            107       1   
8            Amityville Village        9519              9       0   
9                     Amsterdam       18182             30       0   
10               Arcade Village        2053              0       0   
11              Ardsley Village        4523              5       0   
12            Asharoken Village         658              0       0   
13               Attica Village        2522              2       0   
14                       Auburn       27270             96       1   
15                 Avon Village        3333              1       0   
16        Baldwinsville Village        7473              5       0   
17         Ballston Spa Village        5418              8       0   
18                      Batavia       15374             57       0   
19                 Bath Village        5733             15       0   
20                 Bedford Town       17627              5       0   
21               Bethlehem Town       34243             13       0   
22                   Binghamton       46304            284       3   
23                  Black River        1410              0       0   
24          Blooming Grove Town       12001              8       0   
25              Bolivar Village        1022              1       0   
26                  Bolton Town        2325              2       0   
27            Boonville Village        2054              1       0   
28                   Brant Town        2065              4       0   
29                     Brewster        2358              0       0   
..                          ...         ...            ...     ...   
318         Tupper Lake Village        3642              3       0   
319         Tuxedo Park Village         615              0       0   
320                 Ulster Town       12195             14       0   
321                       Utica       61686            361       7   
322                 Vestal Town       27980             16       0   
323              Walden Village        6887             18       1   
324               Wallkill Town       27940             40       0   
325              Walton Village        3012              6       0   
326              Warsaw Village        3453              6       0   
327     Washingtonville Village        5828              2       0   
328  Waterford Town and Village        8392              4       0   
329            Waterloo Village        5175             11       0   
330                   Watertown       28179            109       0   
331                  Watervliet       10245             26       1   
332        Watkins Glen Village        1862              6       0   
333             Waverly Village        4337              5       0   
334    Webster Town and Village       43777             21       0   
335           Weedsport Village        1797              0       0   
336          Wellsville Village        4574             11       0   
337   Westhampton Beach Village        1732              2       0   
338            West Seneca Town       44821             35       1   
339           Whitehall Village        2604              6       0   
340                White Plains       57559             78       1   
341          Whitesboro Village        3733              4       0   
342             Whitestown Town        9141              0       0   
343               Woodbury Town      

In [367]:
#Double Check there is no missing data

def num_missing(x):
  return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
print(crime_df.apply(num_missing, axis=0))

Missing values per column:
City                   0
Population             0
Violent crime          0
Murder                 0
Rape revised           0
Rape legacy            0
Robbery                0
Aggravated assault     0
Property crime         0
Burglary               0
Larceny                0
Motor vehicle theft    0
Arson                  0
Rape combined          0
dtype: int64


In [368]:
#look at data for outliers
print(crime_df.median(), crime_df.mean(), crime_df.min(), crime_df.max(), crime_df.std())

Population            7233.500
Violent crime            6.000
Murder                   0.000
Rape revised             0.000
Rape legacy              0.000
Robbery                  1.000
Aggravated assault       4.000
Property crime         112.500
Burglary                17.500
Larceny                 94.000
Motor vehicle theft      2.000
Arson                    0.000
Rape combined            0.000
dtype: float64 Population            40037.632
Violent crime           201.595
Murder                    1.566
Rape revised              0.000
Rape legacy               5.865
Robbery                  72.902
Aggravated assault      121.261
Property crime          792.606
Burglary                119.684
Larceny                 637.017
Motor vehicle theft      35.905
Arson                     1.006
Rape combined             5.865
dtype: float64 City                   Adams Village
Population                       526
Violent crime                      0
Murder                             0
Rap

In [369]:
#Something is putting our data out of whack. Let's sort and see
print(crime_df.sort_values('Population', ascending=False))

                         City  Population  Violent crime  Murder  \
216                  New York     8396126          52384     335   
35                    Buffalo      258789           3249      47   
272                 Rochester      210562           2107      42   
346                   Yonkers      199134           1036       6   
310                  Syracuse      143834           1192      21   
7                Amherst Town      118296            107       1   
3                      Albany       97956            791       8   
126               Greece Town       96667            151       0   
267               Ramapo Town       87204             57       2   
57            Clarkstown Town       80705             65       0   
214              New Rochelle       78800            175       0   
53           Cheektowaga Town       78361            158       0   
62               Colonie Town       78215             63       0   
206              Mount Vernon       68071       

In [370]:
#Let's Remove New York since it far outnumbers other data
crime_df = crime_df[crime_df.City != 'New York']

In [371]:
sns.heatmap(crime_df.corr())

In [372]:
#A closer look at their relationships (linear?)
g = sns.pairplot(crime_df)

In [373]:
#Let's take a look at the R Square for each
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Population']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 0.03389004]]

Intercept: 
 [-155.02048926]

R-squared:
0.787222200252


In [374]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Violent crime']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 4.12734676]]

Intercept: 
 [ 174.37729304]

R-squared:
0.891800824085


In [375]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Murder']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 255.90303002]]

Intercept: 
 [ 230.88289249]

R-squared:
0.84113570003


In [376]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Rape combined']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 91.37351408]]

Intercept: 
 [ 141.12393493]

R-squared:
0.900286310393


In [377]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Robbery']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 10.16032199]]

Intercept: 
 [ 204.21326707]

R-squared:
0.8702811139


In [378]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Aggravated assault']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 7.56558965]]

Intercept: 
 [ 158.30480899]

R-squared:
0.887266571811


In [379]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Burglary']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 3.81281885]]

Intercept: 
 [ 110.5699272]

R-squared:
0.953763903665


In [380]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Larceny']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 1.43843736]]

Intercept: 
 [-44.33231884]

R-squared:
0.989293348046


In [381]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Arson']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 81.19887115]]

Intercept: 
 [ 303.8512827]

R-squared:
0.384186931449


In [384]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Motor vehicle theft']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 14.5036548]]

Intercept: 
 [ 174.2161472]

R-squared:
0.900644194138


In [385]:
#Let's pick variables that are fairly independent of each other that provide a high R-Squared

# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime_df['Property crime'].values.reshape(-1, 1)
X = crime_df[['Population', 'Murder', 'Larceny']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ -4.63663080e-04   5.00316853e+01   1.22569710e+00]]

Intercept: 
 [-3.60422261]

R-squared:
0.996660458964
